In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense, Input, Add
from keras.models import Sequential, Model, load_model
from Source import golay

Using TensorFlow backend.


In [2]:
G = golay.make_G()

In [3]:
def awgn(x_input, reqSNR):
    sig_power = 1
    
    noise_power = np.sqrt(sig_power / (2*reqSNR))
    noise = noise_power * np.random.normal(size = x_input.shape)
    
    return x_input + noise

In [4]:
codeword_size = 10**6

m = np.random.randint(2, size = (codeword_size, 12))
codeword = np.dot(m,G) % 2

odd = np.where(codeword.sum(axis=1) % 2 == 1)
even_check = np.zeros((codeword_size, 1), dtype='int')
even_check[odd] = 1
codeword = np.concatenate((even_check, codeword), axis=1)
codeword = np.where(codeword > 0 , -1, 1)

In [5]:
Eb_No = 10**(3/10)
Es_No = Eb_No * (12/24)
codeword_noise = awgn(codeword, Es_No)

In [6]:
input_layer = Input(shape = (24,))

line1_Dense1 = Dense(24, activation='elu')(input_layer)
line1_Dense2 = Dense(24, activation='elu')(line1_Dense1)         
line1_Dense3 = Dense(24, activation='elu')(line1_Dense2)
line1_output_layer = Dense(24, activation='tanh')(line1_Dense3)

line2_Dense1 = Dense(48, activation='elu')(input_layer)
line2_Dense2 = Dense(96, activation='elu')(line2_Dense1)         
line2_Dense3 = Dense(48, activation='elu')(line2_Dense2)
line2_output_layer = Dense(24, activation='tanh')(line2_Dense3)

line3_Dense1 = Dense(72, activation='elu')(input_layer)
line3_Dense2 = Dense(216, activation='elu')(line3_Dense1)         
line3_Dense3 = Dense(72, activation='elu')(line3_Dense2)
line3_output_layer = Dense(24, activation='tanh')(line3_Dense3)

line4_Dense1 = Dense(96, activation='elu')(input_layer)
line4_Dense2 = Dense(384, activation='elu')(line4_Dense1)         
line4_Dense3 = Dense(96, activation='elu')(line4_Dense2)
line4_output_layer = Dense(24, activation='tanh')(line4_Dense3)

line5_Dense1 = Dense(120, activation='elu')(input_layer)
line5_Dense2 = Dense(600, activation='elu')(line5_Dense1)         
line5_Dense3 = Dense(120, activation='elu')(line5_Dense2)
line5_output_layer = Dense(24, activation='tanh')(line5_Dense3)

line6_Dense1 = Dense(144, activation='elu')(input_layer)
line6_Dense2 = Dense(864, activation='elu')(line6_Dense1)         
line6_Dense3 = Dense(144, activation='elu')(line6_Dense2)
line6_output_layer = Dense(24, activation='tanh')(line6_Dense3)

add_layer = Add()([line1_output_layer, line2_output_layer, line3_output_layer, 
                   line4_output_layer, line5_output_layer, line6_output_layer])

output_layer = Dense(24, activation='linear')(add_layer)

model = Model(inputs = input_layer, outputs = output_layer)

model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
model.summary()
model.save('model/(24,12)_p6.h5')

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 24)           0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 24)           600         input_1[0][0]                    
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 48)           1200        input_1[0][0]                    
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 72)           1800        input_1[0][0]                    
____________________________________________________________________________________________

In [ ]:
model.fit(codeword_noise, codeword, 
          epochs=100,
          batch_size=1000,
          shuffle=True)

In [9]:
model.save('model/(24,12)_p6.h5')